In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate(prompt,max_tokens=250, temperature=0):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response["choices"][0]['text']

In [2]:
from pyserini.search import SimpleSearcher
import json

searcher = SimpleSearcher('./data/strategyqa_index_1')

In [3]:
test_set = json.load(open("/datadrive/multidocqa/data/strategyqa_reranked.json",'r'))
len(test_set)

229

In [4]:
import json
import argparse
import collections
import numpy as np
import os
import re
import string
import sys
import unicodedata

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  def remove_accents(input_str):
      nfkd_form = unicodedata.normalize('NFKD', input_str)
      only_ascii = nfkd_form.encode('ASCII', 'ignore')
      return only_ascii.decode("utf-8")

  return white_space_fix(remove_articles(remove_punc(lower(remove_accents(s)))))

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5')

2022-07-14 20:25:25.036525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-14 20:25:25.036567: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
explained_train = json.load(open("data/explained_iirc_boolean.json",'r'))
explained_train = [a for a in explained_train if a['answer']["type"]=="binary" and "explanation" in a]
train_embeddings = model.encode([a['question'] for a in explained_train])
len(explained_train)

500

In [ ]:
explained_train = json.load(open("data/qasper_explanation.json","r"))
explained_train = [a for a in explained_train if a['answer'].lower() in ["no","yes"]]
train_embeddings = model.encode([a['question'] for a in explained_train])
len(explained_train)

199

In [7]:
from transformers import GPT2Tokenizer
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def generate_answer(documents, question,kshot=2):
    prompt="For each example, use the documents to create an \"Answer\" and an \"Explanation\" to the \"Question\". Just answer yes or no.\n\nExample 1:\n\n[Document 1]: Title: San Tropez (song). Content: \"San Tropez\" is the fourth track from the album Meddle by the band Pink Floyd. This song was one of several to be considered for the band's \"best of\" album, Echoes: The Best of Pink Floyd.\n    \n[Document 2]: Title: French Riviera. Content: The French Riviera (known in French as the Côte d'Azur [kot daˈzyʁ]; Occitan: Còsta d'Azur [ˈkɔstɔ daˈzyɾ]; literal translation \"Azure Coast\") is the Mediterranean coastline of the southeast corner of France. There is no official boundary, but it is usually considered to extend from Cassis, Toulon or Saint-Tropez on the west to Menton at the France–Italy border in the east, where the Italian Riviera joins. The coast is entirely within the Provence-Alpes-Côte d'Azur (Région Sud) region of France. The Principality of Monaco is a semi-enclave within the region, surrounded on three sides by France and fronting the Mediterranean.\n    \n[Document 3]: Title: Moon Jae-in. Content: Moon also promised transparency in his presidency, moving the presidential residence from the palatial and isolated Blue House to an existing government complex in downtown Seoul.\n\n[Document 4]: Title: Saint-Tropez. Content: Saint-Tropez (US: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, French: [sɛ̃ tʁɔpe]; Occitan: Sant-Tropetz , pronounced [san(t) tʀuˈpes]) is a town on the French Riviera, 68 kilometres (42 miles) west of Nice and 100 kilometres (62 miles) east of Marseille in the Var department of the Provence-Alpes-Côte d'Azur region of Occitania, Southern France.\n\nQuestion: Did Pink Floyd have a song about the French Riviera?\n\nExplanation: According to [Document 1], \"San Tropez\" is a song by Pink Floyd about the French Riviera. This is further supported by [Document 4], which states that Saint-Tropez is a town on the French Riviera. Therefore, the answer is yes\n\nAnswer: yes.\n\n"

    for i, document in enumerate(documents):
        # prompt += "[Document {0}]: Title: {2}. Content: {1}\n\n".format(i+1, document['contents'], document['title'])
        prompt += "[Document {0}]: {1}\n\n".format(i+1, document['contents'])

    prompt+= "Question: {0}\n\nExplanation: According".format(question)

    tokens = tokenizer.tokenize(prompt)
    if len(tokens)+266 > 4000:
        return generate_answer(documents[:-1], question)

    to_return = {
        "prompt": prompt,
        "documents": documents,
        "retry": False,
        "tokens": len(tokens)
    }

    res = generate(prompt)

    to_return['res'] = res

    if "answer:" not in res.lower():
        to_return['retry'] = True
        prompt = prompt+" According"+res+"\nAnswer:"
        to_return['new_prompt'] = prompt
        tokens = tokenizer.tokenize(prompt)
        to_return['tokens'] = len(tokens)
        res = " According"+res+"\nAnswer:"+ generate(prompt,max_tokens=10)
        to_return['res']= res
    
    explanation = res.lower().split("answer:")[0]
    to_return['explanation'] = explanation
    answer = res.lower().split("answer:")[1]
    to_return['answer'] = answer
    to_return['evidence'] = list(set([d['m_id'] for d in documents]))

    return to_return